In [2]:
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 9.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.7/325.7 kB 10.6 MB/s eta 0:00:00


In [5]:
# 📊 06_visual_analysis.ipynb
# Tento notebook slouží k vizualizaci predikcí, zisků a sektorových rozdílů na základě trénovaných modelů

import os
import torch
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import pickle
import sys

from collections import defaultdict

# Cesta ke skriptům
sys.path.append(os.path.abspath("../scripts"))
from utils import Normalizer

# Nastavení cest
DATA_PATH = "../data/preprocessed_datasets.pkl"
MODEL_DIR = "../results"

# Vyber tickery, které chceme vizualizovat
SELECTED_TICKERS = ["MSFT", "AAPL", "META"]

# Definice modelu
class LSTMModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.3):
        super().__init__()
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers,
                                  dropout=dropout, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out).squeeze(-1)

# Načtení datasetů
with open(DATA_PATH, "rb") as f:
    vsechny_datasety = pickle.load(f)

# Uložení predikcí a zisků
viz_data = []

for data in vsechny_datasety:
    ticker = data['ticker']
    if ticker not in SELECTED_TICKERS:
        continue

    x_val = torch.tensor(data['x_val'], dtype=torch.float32)
    y_val = np.array(data['y_val'])
    norm = data['target_normalizer']
    model_path = os.path.join(MODEL_DIR, f"best_model_{ticker}.pth")

    model = LSTMModel(input_size=x_val.shape[2])
    model.load_state_dict(torch.load(model_path, weights_only=True))
    model.eval()

    with torch.no_grad():
        preds = model(x_val).numpy()

    pred_prices = norm.inverse_transform(preds.reshape(-1, 1)).flatten()
    true_prices = norm.inverse_transform(y_val.reshape(-1, 1)).flatten()

    profits = []
    for today, pred, tomorrow in zip(true_prices[:-1], pred_prices[:-1], true_prices[1:]):
        if pred > today:
            profits.append(tomorrow - today)
        else:
            profits.append(0)

    cumulative_profit = np.cumsum(profits)

    viz_data.append({
        "ticker": ticker,
        "pred": pred_prices,
        "actual": true_prices,
        "profits": profits,
        "cumulative": cumulative_profit
    })

# Vizuální zobrazení
for d in viz_data:
    ticker = d['ticker']

    # Predikce vs skutečnost
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=d['actual'], mode='lines', name='Skutečná cena'))
    fig.add_trace(go.Scatter(y=d['pred'], mode='lines', name='Predikce'))
    fig.update_layout(title=f"{ticker}: Predikce vs. skutečnost", xaxis_title="Den", yaxis_title="Cena [USD]")
    fig.write_html(f"../results/{ticker}_predikce.html")

    # Kumulativní zisk
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(y=d['cumulative'], mode='lines', name='Kumulativní zisk'))
    fig2.update_layout(title=f"{ticker}: Simulovaný kumulativní zisk", xaxis_title="Den", yaxis_title="Zisk [USD]")
    fig2.write_html(f"../results/{ticker}_zisk.html")

# Porovnání sektorů
df = pd.read_csv("../results/profit_simulation_summary.csv")

# Zajistí, že se do průměru berou pouze číselné hodnoty
df_numeric = df[["sector", "total_profit"]]
df_grouped = df_numeric.groupby("sector", as_index=False).mean()

# Vytvoření grafu
fig_sector = px.bar(
    df_grouped,
    x="sector",
    y="total_profit",
    title="Průměrný simulovaný zisk podle sektorů",
    labels={"total_profit": "Průměrný zisk [USD]", "sector": "Sektor"}
)

# Uložení
fig_sector.write_html("../results/sektory_zisk.html")
